<a href="https://colab.research.google.com/github/racoope70/exploratory_daytrading/blob/main/ppo_alpaca_paper_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install alpaca-trade-api
!pip install alpaca-trade-api python-dotenv ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 7.7 MB/s eta 0:00:00
  Created wheel for msgpack: filename=msgpack-1.0.3-cp312-cp312-linux_x86_64.whl size=15688 sha256=06a607fd1df93540f9f7b59fbe1c78819b5d452e76e82f55f28b6d8ad1e374bb
  Stored in directory: /root/.cache/pip/wheels/ba/bd/3f/f043e8f634db9c90ae128d631f43ae9990eef01274a63291f9
  Created wheel for websockets: filename=websockets-10.4-cp312-cp312-linux_x86_64.whl size=107332 sha256=9f63dd2a4bd8647f982da714a1af84f36c8a8d8b1dc663ff138e61d9f9b398fe
  Stored in directory: /root/.cache/pip/wheels/80/cf/6d/5d7e4c920cb41925a17

In [2]:
# Clean any partials
!pip uninstall -y stable-baselines3 shimmy gymnasium gym autorom AutoROM.accept-rom-license ale-py

# Install the compatible trio (no [extra] to avoid Atari deps)
!pip install "gymnasium==0.29.1" "shimmy==1.3.0" "stable-baselines3==2.3.0"

# Your other libs (safe to keep separate)
!pip install alpaca-trade-api ta python-dotenv gym-anytrading


Found existing installation: gymnasium 1.2.0
Uninstalling gymnasium-1.2.0:
  Successfully uninstalled gymnasium-1.2.0
Found existing installation: gym 0.25.2
Uninstalling gym-0.25.2:
  Successfully uninstalled gym-0.25.2
Found existing installation: ale-py 0.11.2
Uninstalling ale-py-0.11.2:
  Successfully uninstalled ale-py-0.11.2
Reason for being yanked: Loading broken with PyTorch 1.13
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.1.2 requires ale-py>=0.10.1, which is not installed.
dopamine-rl 4.1.2 requires gym<=0.25.2, which is not installed.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.1 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
# Install if not already
!pip install python-dotenv

In [4]:
import os, json, math, time, typing, numpy as np
import warnings
from datetime import datetime, timezone
from pathlib import Path

import pandas as pd
import ta

from alpaca_trade_api.rest import REST, TimeFrame, APIError

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.vec_env.vec_normalize import VecNormalize

# Colab Drive (optional)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Use a PPO-specific output dir (rename from LightGBM)
RESULTS_DIR = "/content/drive/MyDrive/Alpaca_Results/PPO"
os.makedirs(RESULTS_DIR, exist_ok=True)

warnings.filterwarnings("ignore")


Mounted at /content/drive


In [5]:
from google.colab import files
uploaded = files.upload()  # Prompts you to upload the .env file for the key

Saving Alpaca_keys.env.txt to Alpaca_keys.env.txt


In [6]:
import os

# Rename to .env so dotenv can recognize it
os.rename("Alpaca_keys.env.txt", ".env")

In [7]:
from google.colab import files
uploaded = files.upload()  # Upload: model_ABT.txt, features_ABT.txt, scaler_ABT.pkl


Saving ppo_UNH_window3_features.json to ppo_UNH_window3_features.json
Saving ppo_UNH_window3_model_info.json to ppo_UNH_window3_model_info.json
Saving ppo_UNH_window3_model.zip to ppo_UNH_window3_model.zip
Saving ppo_UNH_window3_probability_config.json to ppo_UNH_window3_probability_config.json
Saving ppo_UNH_window3_vecnorm.pkl to ppo_UNH_window3_vecnorm.pkl


In [8]:
# --- One-time imports & logging ---
# Imports + basic logging + Alpaca init
import os, re, json, csv, shutil, logging, pickle, warnings, time, math
from pathlib import Path
from datetime import datetime, timedelta, timezone
from typing import Dict, Optional, Tuple, Any, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame, APIError
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize

load_dotenv(override=True)
logging.basicConfig(level=logging.INFO, format="%(asctime)s | %(levelname)s | %(message)s")

API_KEY    = os.getenv("APCA_API_KEY_ID")     or os.getenv("ALPACA_API_KEY_ID")     or ""
API_SECRET = os.getenv("APCA_API_SECRET_KEY") or os.getenv("ALPACA_API_SECRET_KEY") or ""
BASE_URL   = os.getenv("APCA_API_BASE_URL")   or os.getenv("ALPACA_API_BASE_URL")   or "https://paper-api.alpaca.markets"

def init_alpaca() -> "tradeapi.REST":
    if not API_KEY or not API_SECRET:
        raise RuntimeError("Missing API keys.")
    api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL)
    _ = api.get_account()  # sanity
    return api

if __name__ == "__main__":
    # Optional quick sanity check (kept out of import-time side effects)
    logging.info("Keys loaded: %s %s | BASE_URL=%s", bool(API_KEY), bool(API_SECRET), BASE_URL)
    api = init_alpaca()
    logging.info("Account status: %s", api.get_account().status)

    # run_live(TICKERS)  # uncomment when ready

def sanity_check():
    print("Keys loaded (bools):", bool(API_KEY), bool(API_SECRET), "| BASE_URL:", BASE_URL)
    api = init_alpaca()
    acct = api.get_account()
    print("Account status:", acct.status)

sanity_check()

Keys loaded (bools): True True | BASE_URL: https://paper-api.alpaca.markets
Account status: ACTIVE


In [9]:
from pathlib import Path
for root in ["/content", "/content/drive/MyDrive"]:
    hits = list(Path(root).rglob("ppo_*_model.zip"))
    if hits:
        print("Found in:", root)
        for h in hits: print("  ", h)


Found in: /content
   /content/ppo_UNH_window3_model.zip
   /content/drive/MyDrive/Results_May_2025/ppo_models_master/ppo_CVX_window1_model.zip
   /content/drive/MyDrive/Results_May_2025/ppo_models_master/ppo_GE_window1_model.zip
Found in: /content/drive/MyDrive
   /content/drive/MyDrive/Results_May_2025/ppo_models_master/ppo_CVX_window1_model.zip
   /content/drive/MyDrive/Results_May_2025/ppo_models_master/ppo_GE_window1_model.zip


In [16]:
# --- UNH artifacts + paper-trading runtime config (run BEFORE sanity_check/run_live) ---
import os, shutil, csv
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone
from alpaca_trade_api.rest import TimeFrame, APIError

# --- Env for UNH paper trading (set FIRST) ---
os.environ["ARTIFACTS_DIR"]     = "/content"
os.environ["TICKERS"]           = "UNH"
os.environ["BEST_WINDOW"]       = "3"
os.environ["INF_DETERMINISTIC"] = "1"
os.environ["DRY_RUN"]           = "0"     # "1"=log only (no orders); "0"=place PAPER orders
os.environ["BARS_FEED"]         = "iex"   # paper feed is "iex"; live (if entitled) would be "sip"
# Derived knobs used by other functions
TICKERS          = [t.strip() for t in os.getenv("TICKERS","UNH").split(",") if t.strip()]
BEST_WINDOW_ENV  = (os.getenv("BEST_WINDOW","").strip() or None)

# local knobs (read once from env)
DRY_RUN   = os.getenv("DRY_RUN", "1").lower() in ("1","true","yes")
BARS_FEED = os.getenv("BARS_FEED", "iex").strip()

# --- CSV logging setup (after ARTIFACTS_DIR is set) ---
RESULTS_DIR = Path(os.getenv("RESULTS_DIR", os.getenv("ARTIFACTS_DIR", "/content")))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

def _append_csv_row(path: Path, row: dict):
    write_header = not path.exists()
    with path.open("a", newline="") as f:
        w = csv.DictWriter(f, fieldnames=row.keys())
        if write_header:
            w.writeheader()
        w.writerow(row)

def log_trade(symbol: str,
              bar_time,            # pandas/np datetime or ISO
              signal: int,         # 1=Buy, 0=Sell/Hold
              raw_action: float,   # PPO raw action
              weight: float,       # target portfolio weight
              confidence: float,   # |tanh(raw_action)|
              price: float,        # last price
              equity: float,       # account equity
              dry_run: bool,
              note: str = ""):
    # normalize times → ISO UTC
    try:
        bt_iso = pd.to_datetime(bar_time, utc=True).isoformat()
    except Exception:
        bt_iso = ""
    row = {
        "log_time": datetime.now(timezone.utc).isoformat(),
        "symbol": symbol,
        "bar_time": bt_iso,
        "signal": "BUY" if int(signal) == 1 else "SELL_OR_HOLD",
        "raw_action": float(raw_action) if np.isfinite(raw_action) else "",
        "weight": float(weight) if np.isfinite(weight) else "",
        "confidence": float(confidence) if np.isfinite(confidence) else "",
        "price": float(price) if np.isfinite(price) else "",
        "equity": float(equity) if np.isfinite(equity) else "",
        "dry_run": int(bool(dry_run)),
        "note": note,
    }
    _append_csv_row(RESULTS_DIR / f"trade_log_{symbol}.csv", row)

# --- Ensure the artifact trio sits together in ARTIFACTS_DIR ---
ART_DIR = Path(os.environ["ARTIFACTS_DIR"])
CANDIDATE_DRIVE_ROOTS = [Path("/content/drive/MyDrive"), Path("/content/drive")]

need = [
    "ppo_UNH_window3_model.zip",
    "ppo_UNH_window3_vecnorm.pkl",
    "ppo_UNH_window3_features.json",
]

for name in need:
    dst = ART_DIR / name
    if dst.exists():
        continue
    src = None
    for root in CANDIDATE_DRIVE_ROOTS:
        try:
            src = next(root.rglob(name))
            break
        except StopIteration:
            continue
    if src:
        dst.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, dst)

# --- Quick verification ---
print("ARTIFACTS_DIR =", ART_DIR)
print("Exists:", ART_DIR.exists())
print("Artifacts found:", sorted(p.name for p in ART_DIR.glob("ppo_UNH_window3_*")))
print("DRY_RUN   =", os.getenv("DRY_RUN"))
print("BARS_FEED =", os.getenv("BARS_FEED"))

missing = [n for n in need if not (ART_DIR / n).exists()]
if missing:
    raise FileNotFoundError(
        f"Missing artifacts in {ART_DIR}:\n  " + "\n  ".join(missing)
    )


ARTIFACTS_DIR = /content
Exists: True
Artifacts found: ['ppo_UNH_window3_features.json', 'ppo_UNH_window3_model.zip', 'ppo_UNH_window3_model_info.json', 'ppo_UNH_window3_probability_config.json', 'ppo_UNH_window3_vecnorm.pkl']
DRY_RUN   = 0
BARS_FEED = iex


In [17]:
# === RUN UNH DIAGNOSTIC (paper if DRY_RUN=='0', dry-run if '1') ===
# --- define unh_diagnostic (drop-in) ---
def unh_diagnostic(dry_run=False, timeframe=TimeFrame.Minute, limit=300)

    ticker = "UNH"
    print(f"\nRunning strategy for {ticker}...")

    # snapshot
    try:
        api = init_alpaca()
        positions_start = len(api.list_positions())
        orders_start    = len(api.list_orders(status="open"))
    except Exception as e:
        print(f"Error initializing Alpaca: {e}")
        return

    # load artifacts
    try:
        best = (os.getenv("BEST_WINDOW","").strip() or None)
        picks   = pick_artifacts_for_ticker(ticker, os.getenv("ARTIFACTS_DIR","/content"), best_window=best)
        model   = load_ppo_model(picks["model"])
        vecnorm = load_vecnormalize(picks["vecnorm"]) if picks["vecnorm"] else None
        feats   = load_features(picks["features"])
        print(f"Model artifacts loaded for {ticker}")
    except Exception as e:
        print(f"Could not load model for {ticker}: {e}")
        return

    # figure out how many bars we need (shape-aware)
    try:
        shape = expected_obs_shape(model, vecnorm)
        lookback = int(shape[0]) if (shape is not None and len(shape) == 2) else None
        bars_limit = max(200, (lookback or 0) * 3)

        bars_df = get_recent_bars(api, ticker, limit=bars_limit, timeframe=timeframe)
        min_rows_needed = lookback if lookback is not None else 60
        if len(bars_df) < min_rows_needed:
            print(f"Not enough data for {ticker}: {len(bars_df)} rows (need ≥ {min_rows_needed})")
            bars_df = None
    except Exception as e:
        print(f"Error fetching bars for {ticker}: {e}")
        bars_df = None

    # build obs
    obs = None
    if bars_df is not None and not bars_df.empty:
        try:
            obs, obs_ts = prepare_observation_from_bars(
                bars_df,
                features_hint=feats,
                min_required_rows=min_rows_needed,
                expected_shape=shape,
            )
        except Exception as e:
            print(f"Error preparing observation for {ticker}: {e}")

    # predict
    signal = None
    if obs is not None:
        try:
            target_w, conf, raw = infer_target_weight(model, vecnorm, obs)
            signal = int(target_w > 0.0)  # 1 = Buy, 0 = Sell/Hold
            print(f"Prediction for {ticker}: {signal} (1 = Buy, 0 = Sell)")
        except Exception as e:
            print(f"Prediction error for {ticker}: {e}")

    # optional paper order if market open
    try:
        clock = api.get_clock()
        if not clock.is_open:
            print("Market is closed.")
        else:
            if signal is not None:
                if not dry_run:
                    has_position = False
                    try:
                        pos = api.get_position(ticker)
                        has_position = float(pos.qty) > 0
                    except APIError:
                        has_position = False

                    if signal == 1 and not has_position:
                        api.submit_order(symbol=ticker, qty=1, side='buy', type='market', time_in_force='day')
                        print(f"BUY order submitted for {ticker}")
                    elif signal == 0 and has_position:
                        api.submit_order(symbol=ticker, qty=1, side='sell', type='market', time_in_force='day')
                        print(f"SELL order submitted for {ticker}")
                    else:
                        print(f"No action taken for {ticker}")
                else:
                    print(f"(dry-run) No order submitted for {ticker} — signal={signal}")
    except Exception as e:
        print(f"Trade/clock error for {ticker}: {e}")

    # tiny summary
    try:
        positions_end = len(api.list_positions())
        orders_end    = len(api.list_orders(status="open"))
        print("\n========== SUMMARY ==========")
        print(f"Processed:         1")
        print(f"Models loaded:     1")
        print(f"Predictions made:  {1 if signal is not None else 0}")
        print(f"Orders submitted:  {'(paper)' if not dry_run else 0}")
        print(f"Existing positions (start -> end): {positions_start} -> {positions_end}")
        print(f"Open orders        (start -> end): {orders_start} -> {orders_end}")
        print("=============================")
    except Exception:
        pass
# --- define unh_diagnostic (drop-in) ---
def unh_diagnostic(dry_run=True, timeframe=TimeFrame.Minute, limit=300):
    ticker = "UNH"
    print(f"\nRunning strategy for {ticker}...")

    # snapshot
    try:
        api = init_alpaca()
        positions_start = len(api.list_positions())
        orders_start    = len(api.list_orders(status="open"))
    except Exception as e:
        print(f"Error initializing Alpaca: {e}")
        return

    # load artifacts
    try:
        best = (os.getenv("BEST_WINDOW","").strip() or None)
        picks   = pick_artifacts_for_ticker(ticker, os.getenv("ARTIFACTS_DIR","/content"), best_window=best)
        model   = load_ppo_model(picks["model"])
        vecnorm = load_vecnormalize(picks["vecnorm"]) if picks["vecnorm"] else None
        feats   = load_features(picks["features"])
        print(f"Model artifacts loaded for {ticker}")
    except Exception as e:
        print(f"Could not load model for {ticker}: {e}")
        return

    # figure out how many bars we need (shape-aware)
    try:
        shape = expected_obs_shape(model, vecnorm)
        lookback = int(shape[0]) if (shape is not None and len(shape) == 2) else None
        bars_limit = max(200, (lookback or 0) * 3)

        bars_df = get_recent_bars(api, ticker, limit=bars_limit, timeframe=timeframe)
        min_rows_needed = lookback if lookback is not None else 60
        if len(bars_df) < min_rows_needed:
            print(f"Not enough data for {ticker}: {len(bars_df)} rows (need ≥ {min_rows_needed})")
            bars_df = None
    except Exception as e:
        print(f"Error fetching bars for {ticker}: {e}")
        bars_df = None

    # build obs
    obs = None
    if bars_df is not None and not bars_df.empty:
        try:
            obs, obs_ts = prepare_observation_from_bars(
                bars_df,
                features_hint=feats,
                min_required_rows=min_rows_needed,
                expected_shape=shape,
            )
        except Exception as e:
            print(f"Error preparing observation for {ticker}: {e}")

    # predict
    signal = None
    if obs is not None:
        try:
            target_w, conf, raw = infer_target_weight(model, vecnorm, obs)
            signal = int(target_w > 0.0)  # 1 = Buy, 0 = Sell/Hold
            print(f"Prediction for {ticker}: {signal} (1 = Buy, 0 = Sell)")
        except Exception as e:
            print(f"Prediction error for {ticker}: {e}")

    # optional paper order if market open
    try:
        clock = api.get_clock()
        if not clock.is_open:
            print("Market is closed.")
        else:
            if signal is not None:
                if not dry_run:
                    has_position = False
                    try:
                        pos = api.get_position(ticker)
                        has_position = float(pos.qty) > 0
                    except APIError:
                        has_position = False

                    if signal == 1 and not has_position:
                        api.submit_order(symbol=ticker, qty=1, side='buy', type='market', time_in_force='day')
                        print(f"BUY order submitted for {ticker}")
                    elif signal == 0 and has_position:
                        api.submit_order(symbol=ticker, qty=1, side='sell', type='market', time_in_force='day')
                        print(f"SELL order submitted for {ticker}")
                    else:
                        print(f"No action taken for {ticker}")
                else:
                    print(f"(dry-run) No order submitted for {ticker} — signal={signal}")
    except Exception as e:
        print(f"Trade/clock error for {ticker}: {e}")

    # tiny summary
    try:
        positions_end = len(api.list_positions())
        orders_end    = len(api.list_orders(status="open"))
        print("\n========== SUMMARY ==========")
        print(f"Processed:         1")
        print(f"Models loaded:     1")
        print(f"Predictions made:  {1 if signal is not None else 0}")
        print(f"Orders submitted:  {'(paper)' if not dry_run else 0}")
        print(f"Existing positions (start -> end): {positions_start} -> {positions_end}")
        print(f"Open orders        (start -> end): {orders_start} -> {orders_end}")
        print("=============================")
    except Exception:
        pass


In [18]:
from typing import Any, List, Tuple, Optional
import numpy as np
import pandas as pd

# ---- Column normalization ---------------------------------------------------------------
def normalize_ohlcv_cols(df: pd.DataFrame) -> pd.DataFrame:
    """
    Case-insensitive rename to: Open, High, Low, Close, Adj Close, Volume.
    If Adj Close missing, copy Close.
    """
    cols_ci = {c.lower(): c for c in df.columns}
    want = {
        "Open":      ["open"],
        "High":      ["high"],
        "Low":       ["low"],
        "Close":     ["close", "close*", "last"],
        "Adj Close": ["adj close", "adj_close", "adjclose", "adjusted close"],
        "Volume":    ["volume", "vol"],
    }
    rename = {}
    for final, alts in want.items():
        for a in [final.lower()] + alts:
            if a in cols_ci:
                rename[cols_ci[a]] = final
                break
    out = df.rename(columns=rename).copy()
    if "Adj Close" not in out.columns and "Close" in out.columns:
        out["Adj Close"] = out["Close"]
    return out


# ---- Light wavelet denoiser (safe fallbacks) -------------------------------------------
def denoise_wavelet(series: pd.Series, wavelet: str = "db1", level: int = 2) -> pd.Series:
    """
    Low-pass via wavelets; if pywt missing or params invalid, fallback to a short EMA.
    """
    try:
        import pywt
    except Exception:
        return pd.Series(series).astype(float).ewm(span=5, adjust=False).mean()

    s = pd.Series(series).astype(float).ffill().bfill()
    arr = s.to_numpy()
    try:
        w = pywt.Wavelet(wavelet)
        maxlvl = pywt.dwt_max_level(len(arr), w.dec_len)
        lvl = int(max(0, min(level, maxlvl)))
        if lvl < 1:
            return s
        coeffs = pywt.wavedec(arr, w, mode="symmetric", level=lvl)
        for i in range(1, len(coeffs)):  # zero details, keep approximation
            coeffs[i] = np.zeros_like(coeffs[i])
        rec = pywt.waverec(coeffs, w, mode="symmetric")
        return pd.Series(rec[:len(arr)], index=s.index)
    except Exception:
        return s.ewm(span=5, adjust=False).mean()


# ---- Simple regime labels ---------------------------------------------------------------
def add_regime(df: pd.DataFrame) -> pd.DataFrame:
    df["Vol20"] = df["Close"].pct_change().rolling(20).std()
    df["Ret20"] = df["Close"].pct_change(20)
    vol_hi   = (df["Vol20"] > df["Vol20"].median()).astype(int)
    trend_hi = (df["Ret20"].abs() > df["Ret20"].abs().median()).astype(int)
    df["Regime4"] = vol_hi * 2 + trend_hi  # values in {0,1,2,3}
    return df


# ---- Indicators used at live time (match training set) ---------------------------------
def add_features_live(
    df: pd.DataFrame,
    use_sentiment: bool = False,
    rsi_wilder: bool = True,
    atr_wilder: bool = True,
) -> pd.DataFrame:
    """
    Adds a compact set of indicators (Bollinger, Stoch, ROC, OBV, CCI, EMAs, MACD, RSI, ATR, Volatility).
    Returns a numeric DataFrame with NaN where warmups are needed (to be dropped later).
    """
    df = normalize_ohlcv_cols(df).copy().sort_index()

    # Bollinger
    df["SMA_20"] = df["Close"].rolling(20).mean()
    df["STD_20"] = df["Close"].rolling(20).std()
    df["Upper_Band"] = df["SMA_20"] + 2 * df["STD_20"]
    df["Lower_Band"] = df["SMA_20"] - 2 * df["STD_20"]

    # Stochastic %K (unsmoothed)
    df["Lowest_Low"]   = df["Low"].rolling(14).min()
    df["Highest_High"] = df["High"].rolling(14).max()
    denom = (df["Highest_High"] - df["Lowest_Low"]).replace(0, np.nan)
    df["Stoch"] = ((df["Close"] - df["Lowest_Low"]) / denom) * 100

    # Momentum / volume / CCI
    df["ROC"] = df["Close"].pct_change(10)
    sign = np.sign(df["Close"].diff().fillna(0))
    df["OBV"] = (sign * df["Volume"].fillna(0)).cumsum()

    tp = (df["High"] + df["Low"] + df["Close"]) / 3.0
    sma_tp = tp.rolling(20).mean()
    md = (tp - sma_tp).abs().rolling(20).mean().replace(0, np.nan)
    df["CCI"] = (tp - sma_tp) / (0.015 * md)

    # EMAs + MACD
    df["EMA_10"] = df["Close"].ewm(span=10, adjust=False).mean()
    df["EMA_50"] = df["Close"].ewm(span=50, adjust=False).mean()
    ema12 = df["Close"].ewm(span=12, adjust=False).mean()
    ema26 = df["Close"].ewm(span=26, adjust=False).mean()
    df["MACD_Line"]   = ema12 - ema26
    df["MACD_Signal"] = df["MACD_Line"].ewm(span=9, adjust=False).mean()

    # RSI (Wilder default)
    d = df["Close"].diff()
    gain = d.clip(lower=0)
    loss = (-d.clip(upper=0))
    if rsi_wilder:
        avg_gain = gain.ewm(alpha=1/14, adjust=False).mean()
        avg_loss = loss.ewm(alpha=1/14, adjust=False).mean()
    else:
        avg_gain = gain.rolling(14).mean()
        avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    df["RSI"] = 100 - (100 / (1 + rs))

    # ATR (Wilder default)
    tr = pd.concat([
        (df["High"] - df["Low"]),
        (df["High"] - df["Close"].shift()).abs(),
        (df["Low"]  - df["Close"].shift()).abs(),
    ], axis=1).max(axis=1)
    df["ATR"] = tr.ewm(alpha=1/14, adjust=False).mean() if atr_wilder else tr.rolling(14).mean()

    # Volatility + denoised close
    df["Volatility"] = df["Close"].pct_change().rolling(20).std()
    df["Denoised_Close"] = denoise_wavelet(df["Close"])

    # Regime + optional sentiment + simple “Greeks” proxies
    df = add_regime(df)
    df["SentimentScore"] = (df.get("SentimentScore", 0.0) if use_sentiment else 0.0)
    df["Delta"] = df["Close"].pct_change(1).fillna(0.0)
    df["Gamma"] = df["Delta"].diff().fillna(0.0)

    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    return df


# ---- Artifact feature order (for 1D vector models) -------------------------------------
def compute_art_feat_order(features_hint: Any, df: pd.DataFrame) -> List[str]:
    """
    Resolve the exact feature order from your artifact, keeping only numeric columns present.
    Accepts either a list or a dict with key 'features'.
    """
    if features_hint is None:
        return [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]

    feats = features_hint.get("features", features_hint) if isinstance(features_hint, dict) else list(features_hint)
    drop = {"datetime", "symbol", "target", "return"}
    return [c for c in feats if c not in drop and (c in df.columns) and pd.api.types.is_numeric_dtype(df[c])]


def build_obs_from_row(row: pd.Series, order: List[str]) -> np.ndarray:
    """Row -> 1D float32 vector in the artifact's column order (NaNs -> 0)."""
    vals = []
    for c in order:
        v = row.get(c, np.nan)
        vals.append(0.0 if (pd.isna(v) or v is None or v is False) else float(v))
    return np.array(vals, dtype=np.float32)


# ---- Shape discovery + live columns picking (for 2D models) -----------------------------
def expected_obs_shape(model, vecnorm) -> Optional[tuple]:
    """
    Returns the tuple shape the model expects for a *single* observation,
    e.g. (10, 2). Falls back to None if not available.
    """
    for src in (vecnorm, model):
        try:
            shp = tuple(src.observation_space.shape)  # SB3 exposes this
            if shp:
                return shp
        except Exception:
            pass
    return None  # unknown; caller will use 1D fallback


def _pick_columns_for_channels(features_hint: Any, df: pd.DataFrame, channels: int) -> List[str]:
    """
    Pick which DataFrame columns to use as 'channels' when building a 2D observation.
    Priority: artifact feature list (if present) → common OHLCV columns → other numeric.
    """
    numeric = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    cols: List[str] = []

    if isinstance(features_hint, dict) and "features" in features_hint:
        cand = [c for c in features_hint["features"] if c in df.columns and pd.api.types.is_numeric_dtype(df[c])]
        if len(cand) >= channels:
            cols = cand[:channels]

    if not cols:
        pref = ["Close", "Volume", "Adj Close", "Open", "High", "Low"]
        cols = [c for c in pref if c in numeric]
        cols += [c for c in numeric if c not in cols]
        cols = cols[:channels]

    if len(cols) < channels and cols:
        while len(cols) < channels:
            cols.append(cols[-1])  # duplicate last to fill
    return cols[:channels]


# ---- Observation builder (2D-aware with 1D fallback) ------------------------------------
def prepare_observation_from_bars(
    bars_df: pd.DataFrame,
    features_hint: Any = None,
    min_required_rows: int = 60,
    expected_shape: Optional[tuple] = None,
) -> Tuple[np.ndarray, int]:
    """
    Convert recent OHLCV bars to a single observation.
    If the model expects 2D (lookback, channels), we build a window using raw columns.
    Otherwise we build a 1D feature vector in the artifact's column order.

    Returns: (obs: np.ndarray, obs_timestamp_utc: int)
    """
    feats_df = add_features_live(bars_df).replace([np.inf, -np.inf], np.nan)

    # Timestamp for staleness guard
    ts = pd.Timestamp.utcnow()
    try:
        idx_ts = pd.Timestamp(feats_df.index[-1])
        ts = idx_ts.tz_convert("UTC") if idx_ts.tzinfo else idx_ts.tz_localize("UTC")
    except Exception:
        pass

    # ---- 2D path (e.g., expected_shape == (10, 2)) --------------------------------------
    if expected_shape is not None and len(expected_shape) == 2:
        lookback, channels = int(expected_shape[0]), int(expected_shape[1])
        cols = _pick_columns_for_channels(features_hint, feats_df, channels)

        # Use the raw (or close-to-raw) columns; take tail(lookback) and pad if short
        window_df = feats_df[cols].tail(lookback)
        arr = window_df.to_numpy(dtype=np.float32)

        if arr.shape[0] < lookback:
            pad_rows = lookback - arr.shape[0]
            arr = np.vstack([np.zeros((pad_rows, channels), dtype=np.float32), arr])

        # Guard final shape exactly
        arr = arr[-lookback:, :channels]
        return arr.reshape(lookback, channels), int(ts.timestamp())

    # ---- 1D fallback (artifact feature vector) ------------------------------------------
    order = compute_art_feat_order(features_hint, feats_df)
    if not order:
        raise ValueError("No usable features after resolving artifact order.")
    feats_df = feats_df.dropna(subset=order)

    if len(feats_df) < max(20, min_required_rows):
        raise ValueError(f"Not enough bars to compute features robustly (have {len(feats_df)}).")

    last = feats_df.iloc[-1]
    obs = build_obs_from_row(last, order)
    return obs.astype(np.float32), int(ts.timestamp())


In [19]:
# --- PPO paper-trading setup (Colab) ---
import os, shutil
from pathlib import Path

# 1) Env toggles
os.environ.update({
    "ARTIFACTS_DIR": "/content",
    "TICKERS": "UNH",
    "INF_DETERMINISTIC": "1",  # PPO deterministic inference
    "DRY_RUN": "1",            # no real orders
    "BARS_FEED": "iex",        # paper feed
    # "BEST_WINDOW": "3",      # optional; leave unset if you only have one model
})

# 2) Make sure artifacts are colocated
ART = Path(os.environ["ARTIFACTS_DIR"])
NEED = [
    "ppo_UNH_window3_model.zip",       # or your single model name
    "ppo_UNH_window3_vecnorm.pkl",     # if you have it
    "ppo_UNH_window3_features.json",   # if you saved feature order
]
DRIVE = Path("/content/drive/MyDrive")
for name in NEED:
    dst = ART / name
    if not dst.exists():
        src = next(DRIVE.rglob(name), None)
        if src:
            dst.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(src, dst)

print("ARTIFACTS:", sorted(p.name for p in ART.glob("ppo_UNH*")))

# 3) (Re)define INF_DETERMINISTIC in case main reads after this cell
INF_DETERMINISTIC = os.getenv("INF_DETERMINISTIC", "1").lower() in ("1","true","yes")

ARTIFACTS: ['ppo_UNH_window3_features.json', 'ppo_UNH_window3_model.zip', 'ppo_UNH_window3_model_info.json', 'ppo_UNH_window3_probability_config.json', 'ppo_UNH_window3_vecnorm.pkl']


In [20]:
# --- Put these runtime knobs near the top of your script (before the picker) ---
ARTIFACTS_DIR   = os.getenv("ARTIFACTS_DIR", "/content")
BEST_WINDOW_ENV = (os.getenv("BEST_WINDOW", "").strip() or None)  # OPTIONAL

# --- Helper stays the same (optional: used when multiple windows are present) ---
def _extract_window_idx(path: Path) -> Optional[int]:
    m = re.search(r"_window(\d+)_", path.stem, re.IGNORECASE)
    if not m: return None
    try:
        return int(m.group(1))
    except Exception:
        return None

# --- REPLACE your pick_artifacts_for_ticker with this version ---
def pick_artifacts_for_ticker(
    ticker: str,
    artifacts_dir: str,
    best_window: Optional[str] = None  # <- no global in the default
) -> Dict[str, Optional[Path]]:
    p = Path(artifacts_dir)
    if not p.exists():
        raise FileNotFoundError(f"Artifacts directory not found: {p.resolve()}")

    # Look for common model filenames
    models = sorted(p.glob(f"ppo_{ticker}_window*_model.zip"))
    if not models:
        # fallbacks if you didn't include the window in the filename
        models = sorted(p.glob(f"ppo_{ticker}_model.zip")) or sorted(p.glob(f"*{ticker}*model.zip"))
        if not models:
            found = "\n".join(f" - {x}" for x in p.rglob("*model.zip"))
            raise FileNotFoundError(f"No PPO model for {ticker} in {p.resolve()}.\nSeen:\n{found or ' (none)'}")

    # If an env-provided BEST_WINDOW is set, try to honor it; otherwise pick sensibly
    chosen: Optional[Path] = None
    if best_window:
        chosen = next((m for m in models if f"_window{best_window}_" in m.stem), None)
        if chosen is None:
            logging.warning("BEST_WINDOW=%s not found; falling back to best available.", best_window)

    if chosen is None:
        # If multiple windows exist, pick the one with the largest window index; else last sorted
        with_idx = [(m, _extract_window_idx(m)) for m in models]
        with_idx = [(m, w) for (m, w) in with_idx if w is not None]
        chosen = max(with_idx, key=lambda t: t[1])[0] if with_idx else models[-1]

    base   = chosen.stem.replace("_model", "")
    vecnorm = next(iter(sorted(p.glob(base + "_vecnorm.pkl"))), None)
    feats   = next(iter(sorted(p.glob(base + "_features.json"))), None)

    logging.info(f"[{ticker}] model={chosen.name} | vecnorm={bool(vecnorm)} | features={bool(feats)}")
    return {"model": chosen, "vecnorm": vecnorm, "features": feats}

def load_vecnormalize(path: Optional[Path]):
    if path is None:
        return None
    with open(path, "rb") as f:
        return pickle.load(f)

def load_features(path: Optional[Path]):
    if path is None:
        return None
    with open(path, "r") as f:
        return json.load(f)

def load_ppo_model(model_path: Path):
    return PPO.load(str(model_path))

# ===== Alpaca helpers ====================================================================

def get_recent_bars(api, symbol: str, limit: int = 200, timeframe=TimeFrame.Minute) -> pd.DataFrame:
    """
    Fetch recent bars and return a DataFrame with index=timestamp and
    columns: Open, High, Low, Close, Volume. Reads BARS_FEED at call time.
    If empty, retries with a 5-day start/end window.
    """
    def _as_df(bars):
        if hasattr(bars, "df"):
            df = bars.df.copy()
            if not df.empty:
                if isinstance(df.index, pd.MultiIndex):
                    df = df.xs(symbol, level=0)
                df.index = pd.to_datetime(df.index, utc=True)
                df = df.rename(columns={"open":"Open","high":"High","low":"Low","close":"Close","volume":"Volume"})
                return df[["Open","High","Low","Close","Volume"]].sort_index()
            return pd.DataFrame(columns=["Open","High","Low","Close","Volume"])

        # list-like fallback
        rows = []
        for b in bars:
            ts = pd.to_datetime(getattr(b, "t", None), utc=True)
            rows.append({
                "timestamp": ts,
                "Open":   float(getattr(b, "o", getattr(b, "open",  np.nan))),
                "High":   float(getattr(b, "h", getattr(b, "high",  np.nan))),
                "Low":    float(getattr(b, "l", getattr(b, "low",   np.nan))),
                "Close":  float(getattr(b, "c", getattr(b, "close", np.nan))),
                "Volume": float(getattr(b, "v", getattr(b, "volume",np.nan))),
            })
        return pd.DataFrame(rows).set_index("timestamp").sort_index()

    feed = os.getenv("BARS_FEED", "").strip()  # read at call time
    try:
        logging.info(f"[{symbol}] fetching {limit} {timeframe} bars (feed='{feed or 'default'}')")
        if feed:
            bars = api.get_bars(symbol, timeframe, limit=limit, feed=feed)
        else:
            bars = api.get_bars(symbol, timeframe, limit=limit)
        df = _as_df(bars)
        if not df.empty:
            return df
    except Exception as e:
        logging.warning(f"[{symbol}] get_bars(limit) failed: {e}")

    # Fallback: explicit 5-day window
    try:
        end   = datetime.utcnow().replace(microsecond=0).isoformat() + "Z"
        start = (datetime.utcnow() - timedelta(days=5)).replace(microsecond=0).isoformat() + "Z"
        logging.info(f"[{symbol}] retry with window start={start} end={end} (feed='{feed or 'default'}')")
        if feed:
            bars = api.get_bars(symbol, timeframe, start=start, end=end, feed=feed)
        else:
            bars = api.get_bars(symbol, timeframe, start=start, end=end)
        return _as_df(bars)
    except Exception as e:
        logging.warning(f"[{symbol}] get_bars(start/end) failed: {e}")
        return pd.DataFrame(columns=["Open","High","Low","Close","Volume"])

def get_account_equity(api) -> float:
    return float(api.get_account().equity)

def get_position(api, symbol: str):
    try:
        return api.get_position(symbol)
    except Exception:
        return None

def get_position_qty(api, symbol: str) -> int:
    pos = get_position(api, symbol)
    if not pos:
        return 0
    try:
        return int(float(pos.qty))
    except Exception:
        return 0

def get_last_price(api, symbol: str) -> float:
    try:
        bars = api.get_bars(symbol, TimeFrame.Minute, limit=1)
        if len(bars) > 0:
            b = bars[0]
            close = getattr(b, "c", getattr(b, "close", None))
            if close is not None:
                return float(close)
    except Exception as e:
        logging.warning(f"[{symbol}] get_last_price via bars failed: {e}")
    pos = get_position(api, symbol)
    if pos:
        try:
            return float(pos.avg_entry_price)
        except Exception:
            pass
    return float("nan")

def cancel_open_symbol_orders(api, symbol: str):
    try:
        for o in api.list_orders(status="open"):
            if o.symbol == symbol:
                api.cancel_order(o.id)
    except Exception as e:
        logging.warning(f"[{symbol}] cancel orders failed: {e}")

def market_order_to_qty(api, symbol: str, side: str, qty: int):
    if qty <= 0:
        return None
    if DRY_RUN:
        logging.info(f"[DRY_RUN] Would submit {side} {qty} {symbol} (market, day)")
        return None
    try:
        o = api.submit_order(
            symbol=symbol,
            side=side,
            type="market",
            qty=qty,
            time_in_force="day"
        )
        logging.info(f"[{symbol}] Submitted {side} {qty} (market) id={o.id}")
        return o
    except Exception as e:
        logging.error(f"[{symbol}] submit_order failed: {e}")
        return None

# ===== Sizing / Risk =====================================================================
def action_to_weight(action) -> Tuple[float, float, float]:
    a = float(np.array(action).squeeze())
    if a <= 0:
        return 0.0, abs(np.tanh(a)), a
    conf = abs(np.tanh(a))
    if SIZING_MODE == "linear":
        w = WEIGHT_CAP * conf
    else:
        w = 0.0 if conf < CONF_FLOOR else WEIGHT_CAP * (conf - CONF_FLOOR) / (1.0 - CONF_FLOOR)
    return max(0.0, min(WEIGHT_CAP, w)), conf, a

# Optional: avoid 0-share buys for small positive weights
MIN_POS_QTY = 1
def compute_target_qty(equity: float, price: float, target_weight: float) -> int:
    if not np.isfinite(price) or price <= 0:
        return 0
    notional = equity * target_weight
    qty = int(notional // price)  # floor
    if target_weight > 0 and qty == 0:
        return MIN_POS_QTY
    return max(0, qty)
def compute_target_qty_by_cash(equity: float, price: float, target_weight: float, api=None) -> int:
    if not np.isfinite(price) or price <= 0:
        return 0
    cash_cap = float(api.get_account().cash) if api else equity
    notional = min(cash_cap, equity * target_weight)
    qty = int(notional // price)
    if target_weight > 0 and qty == 0 and notional >= price:
        return 1
    return max(0, qty)


def flatten_symbol(api, symbol: str):
    qty = get_position_qty(api, symbol)
    if qty == 0:
        return
    cancel_open_symbol_orders(api, symbol)
    side = "sell" if qty > 0 else "buy"
    market_order_to_qty(api, symbol, side, abs(qty))

def rebalance_to_weight(api, symbol: str, equity: float, target_weight: float):
    if target_weight <= 0:
        flatten_symbol(api, symbol)
        return
    price    = get_last_price(api, symbol)
    if not np.isfinite(price) or price <= 0:
        logging.warning(f"[{symbol}] Price unavailable; skipping rebalance this cycle.")
        return
    have_qty = get_position_qty(api, symbol)
    want_qty = compute_target_qty_by_cash(equity, price, target_weight, api)
    delta    = want_qty - have_qty

    delta    = want_qty - have_qty
    if delta == 0:
        return
    side = "buy" if delta > 0 else "sell"
    market_order_to_qty(api, symbol, side, abs(delta))

# ===== Simple TP/SL (optional) ============================================================
def check_tp_sl_and_maybe_flatten(api, symbol: str) -> bool:
    if TAKE_PROFIT_PCT <= 0 and STOP_LOSS_PCT <= 0:
        return False
    pos = get_position(api, symbol)
    if not pos:
        return False
    try:
        plpc = float(pos.unrealized_plpc)  # +0.031 = +3.1%
    except Exception:
        return False
    if TAKE_PROFIT_PCT > 0 and plpc >= TAKE_PROFIT_PCT:
        logging.info(f"[{symbol}] TP hit ({plpc:.4f} >= {TAKE_PROFIT_PCT:.4f}). Flattening.")
        flatten_symbol(api, symbol)
        return True
    if STOP_LOSS_PCT > 0 and plpc <= -abs(STOP_LOSS_PCT):
        logging.info(f"[{symbol}] SL hit ({plpc:.4f} <= {-abs(STOP_LOSS_PCT):.4f}). Flattening.")
        flatten_symbol(api, symbol)
        return True
    return False

# ===== Inference ==========================================================================
def infer_target_weight(model: PPO, vecnorm: Optional[VecNormalize], obs: np.ndarray) -> Tuple[float, float, float]:
    x = obs
    if vecnorm is not None and hasattr(vecnorm, "normalize_obs"):
        try:
            x = vecnorm.normalize_obs(x)
        except Exception as e:
            logging.warning(f"VecNormalize.normalize_obs failed; using raw obs. Err: {e}")
    action, _ = model.predict(x, deterministic=INF_DETERMINISTIC)
    return action_to_weight(action)

# ===== Single-symbol step ================================================================
def run_live_once_for_symbol(api, symbol: str, model: PPO, vecnorm: Optional[VecNormalize], features_hint: Optional[dict] = None):
    # Discover the expected shape (e.g., (lookback, channels))
    shape = expected_obs_shape(model, vecnorm)

    # 1) Get fresh bars → features → obs
    bars_df = get_recent_bars(api, symbol, limit=200, timeframe=TimeFrame.Minute)
    if bars_df.empty:
        logging.warning(f"[{symbol}] No recent bars; skipping.")
        return

    obs, obs_ts = prepare_observation_from_bars(
        bars_df,
        features_hint=features_hint,
        min_required_rows=60,
        expected_shape=shape,   # <-- IMPORTANT
    )

    # 2) Staleness guard
    if utc_ts(now_utc()) - obs_ts > STALE_MAX_SEC:
        logging.warning(f"[{symbol}] Observation stale (> {STALE_MAX_SEC}s). Skipping.")
        return

    # 3) Optional TP/SL pre-check
    if check_tp_sl_and_maybe_flatten(api, symbol):
        return

    # 4) Inference → target weight
    target_w, conf, raw = infer_target_weight(model, vecnorm, obs)
    eq = get_account_equity(api)
    logging.info(f"[{symbol}] action={raw:.4f} conf={conf:.3f} → target_w={target_w:.3f} | equity=${eq:,.2f}")
    bar_time = bars_df.index[-1] if not bars_df.empty else pd.NaT
    price    = float(bars_df["Close"].iloc[-1]) if not bars_df.empty else get_last_price(api, symbol)
    log_trade(symbol, bar_time, int(target_w > 0.0), raw, target_w, conf, price, eq, DRY_RUN, note="live")

    # 5) Rebalance
    rebalance_to_weight(api, symbol, eq, target_w)


# ===== Live loop =========================================================================
def run_live(tickers: List[str]):
    api = init_alpaca()

    # Load artifacts per ticker
    per_ticker: Dict[str, Tuple[PPO, Optional[VecNormalize], Optional[dict]]] = {}
    for t in tickers:
        picks = pick_artifacts_for_ticker("UNH", ARTIFACTS_DIR, best_window=BEST_WINDOW_ENV)
        model      = load_ppo_model(picks["model"])
        vecnorm    = load_vecnormalize(picks["vecnorm"]) if picks["vecnorm"] else None
        feat_order = load_features(picks["features"])
        per_ticker[t] = (model, vecnorm, feat_order)

    logging.info(f"Starting live execution for: {tickers}")
    last_exec_at = now_utc() - timedelta(minutes=COOLDOWN_MIN)

    while True:
        try:
            if not ensure_market_open(api):
                logging.info("Market closed. Sleeping 30s.")
                time.sleep(30)
                continue

            if (now_utc() - last_exec_at) < timedelta(minutes=COOLDOWN_MIN):
                time.sleep(5)
                continue

            for t in tickers:
                model, vecnorm, feat_hint = per_ticker[t]
                run_live_once_for_symbol(api, t, model, vecnorm, features_hint=feat_hint)

            last_exec_at = now_utc()

        except KeyboardInterrupt:
            logging.info("KeyboardInterrupt: stopping live loop.")
            break
        except Exception as e:
            logging.exception(f"Live loop exception: {e}")
            time.sleep(5)

# ===== UNH Diagnostic (friendly prints, works even if market is closed) ===================
def unh_diagnostic(
    dry_run: bool = True,
    timeframe: TimeFrame = TimeFrame.Minute,
    limit: int = 300
):
    """
    Runs a one-off UNH evaluation, printing messages like your example:
      Running strategy for UNH...
      Model artifacts loaded for UNH
      Prediction for UNH: 0 (1 = Buy, 0 = Sell)
      Market is closed.
    """
    ticker = "UNH"
    print(f"\nRunning strategy for {ticker}...")

    # Stats for the summary
    processed = 1
    loaded    = 0
    predicted = 0
    closed    = 0
    data_err  = 0
    model_err = 0
    orders    = 0

    # Connect & snapshot counts
    try:
        api = init_alpaca()
        positions_start = len(api.list_positions())
        orders_start    = len(api.list_orders(status="open"))
    except Exception as e:
        print(f"Error initializing Alpaca: {e}")
        return

    # Load artifacts
    try:
        picks   = pick_artifacts_for_ticker(ticker, ARTIFACTS_DIR, best_window=BEST_WINDOW_ENV)
        model   = load_ppo_model(picks["model"])
        vecnorm = load_vecnormalize(picks["vecnorm"]) if picks["vecnorm"] else None
        feats   = load_features(picks["features"])
        print(f"Model artifacts loaded for {ticker}")
        loaded += 1
    except Exception as e:
        print(f"Could not load model for {ticker}: {e}")
        model_err += 1
        # Summary
        positions_end = len(api.list_positions())
        orders_end    = len(api.list_orders(status="open"))
        print("\n========== SUMMARY ==========")
        print(f"Processed:         {processed}")
        print(f"Models loaded:     {loaded}")
        print(f"Predictions made:  {predicted}")
        print(f"Market closed:     {closed}")
        print(f"Data errors:       {data_err}")
        print(f"Model errors:      {model_err}")
        print(f"Orders submitted:  {orders} (dry_run={dry_run})")
        print(f"Existing positions (start -> end): {positions_start} -> {positions_end}")
        print(f"Open orders        (start -> end): {orders_start} -> {orders_end}")
        print("=============================")
        return

    # Fetch bars (recent)
    # ===== Fetch bars (recent) — shape-aware minimum =====
    try:
        # Discover model's expected obs shape (e.g., (lookback, channels))
        shape = expected_obs_shape(model, vecnorm)
        lookback = int(shape[0]) if (shape is not None and len(shape) == 2) else None

        # Pull enough bars (3× lookback if known; otherwise 200)
        bars_limit = max(200, (lookback or 0) * 3)
        bars_df = get_recent_bars(api, ticker, limit=bars_limit, timeframe=timeframe)

        # Dynamic minimum rows:
        #  - If model is 2D (lookback, channels) -> need at least `lookback` rows
        #  - If model is 1D vector -> keep your normal 60-row requirement (tune if desired)
        min_rows_needed = lookback if lookback is not None else 60
        if len(bars_df) < min_rows_needed:
            print(f"Not enough data for {ticker}: {len(bars_df)} rows (need ≥ {min_rows_needed})")
            data_err += 1
            raise ValueError("insufficient bars")

        # Build an observation (pass the shape so we use the 2D path when available)
        obs, obs_ts = prepare_observation_from_bars(
            bars_df,
            features_hint=feats,
            min_required_rows=min_rows_needed,
            expected_shape=shape,   # <-- critical for (lookback, channels) models
        )
    except Exception as e:
        print(f"Error fetching/processing bars for {ticker}: {e}")
        data_err += 1
        bars_df = None
        obs = None

    # Predict (binary-like, derived from PPO weight)
    signal = None
    if obs is not None:
        try:
            target_w, conf, raw = infer_target_weight(model, vecnorm, obs)
            signal = int(target_w > 0.0)  # 1 = Buy, 0 = Sell/Hold
            print(f"Prediction for {ticker}: {signal} (1 = Buy, 0 = Sell)")
            bar_time = bars_df.index[-1] if (bars_df is not None and not bars_df.empty) else pd.NaT
            price    = float(bars_df["Close"].iloc[-1]) if (bars_df is not None and not bars_df.empty) else get_last_price(api, ticker)
            equity   = get_account_equity(api)
            log_trade(ticker, bar_time, signal, raw, target_w, conf, price, equity, dry_run=True, note="diagnostic")
            predicted += 1
        except Exception as e:
            print(f"Prediction error for {ticker}: {e}")
            model_err += 1

    # Trade execution (dry-run friendly) + market-open info
    try:
        clock = api.get_clock()
        if not clock.is_open:
            print("Market is closed.")
            closed += 1
        else:
            # Only attempt order submission if not dry_run and we had a valid signal
            FORCE_FIRST_BUY = os.getenv("FORCE_FIRST_BUY","0").lower() in ("1","true","yes")

            # Only attempt order submission if not dry_run and we had a valid signal
            if not dry_run and signal is not None:
                FORCE_FIRST_BUY = os.getenv("FORCE_FIRST_BUY","0").lower() in ("1","true","yes")
                has_position = False
                try:
                    pos = api.get_position(ticker)
                    has_position = float(pos.qty) > 0
                except APIError:
                    has_position = False

                if signal == 1 and (FORCE_FIRST_BUY or not has_position):
                    api.submit_order(symbol=ticker, qty=1, side='buy', type='market', time_in_force='day')
                    orders += 1
                    print(f"BUY order submitted for {ticker}")

                if signal == 1 and not has_position:
                    api.submit_order(symbol=ticker, qty=1, side='buy', type='market', time_in_force='day')
                    orders += 1
                    print(f"BUY order submitted for {ticker}")
                elif signal == 0 and has_position:
                    api.submit_order(symbol=ticker, qty=1, side='sell', type='market', time_in_force='day')
                    orders += 1
                    print(f"SELL order submitted for {ticker}")
                else:
                    print(f"No action taken for {ticker}")
            else:
                if signal is not None:
                    print(f"(dry-run) No order submitted for {ticker} — signal={signal}")
    except Exception as e:
        print(f"Trade/clock error for {ticker}: {e}")

    # Final snapshot + summary
    positions_end = len(api.list_positions())
    orders_end    = len(api.list_orders(status="open"))
    print("\n========== SUMMARY ==========")
    print(f"Processed:         {processed}")
    print(f"Models loaded:     {loaded}")
    print(f"Predictions made:  {predicted}")
    print(f"Market closed:     {closed}")
    print(f"Data errors:       {data_err}")
    print(f"Model errors:      {model_err}")
    print(f"Orders submitted:  {orders} (dry_run={dry_run})")
    print(f"Existing positions (start -> end): {positions_start} -> {positions_end}")
    print(f"Open orders        (start -> end): {orders_start} -> {orders_end}")
    print("=============================")

# ===== Utilities: quick sanity ============================================================
def sanity_check(run_single_step: bool = True):
    """Connectivity + artifacts + optional one live step for first ticker."""
    logging.info("Sanity: keys=%s/%s base=%s", bool(API_KEY), bool(API_SECRET), BASE_URL)
    api = init_alpaca()
    acct = api.get_account()
    logging.info("Account status: %s | equity=%s | cash=%s", acct.status, acct.equity, acct.cash)

    if not TICKERS:
        raise RuntimeError("No TICKERS configured.")
    for t in tickers:
      picks = pick_artifacts_for_ticker(t0, ARTIFACTS_DIR, best_window=BEST_WINDOW_ENV)  # not 't'
      logging.info("Artifacts for %s -> %s", t0, {k: (str(v) if v else None) for k,v in picks.items()})

    if run_single_step:
        model   = load_ppo_model(picks["model"])
        vecnorm = load_vecnormalize(picks["vecnorm"]) if picks["vecnorm"] else None
        feat    = load_features(picks["features"])
        run_live_once_for_symbol(api, t0, model, vecnorm, features_hint=feat)
        print("One-step OK for", t0)


In [21]:
# ================================
# Live-trade-only plotting add-on
#   • plots only when fills exist
#   • quiet (no chart) if no fills
# ================================
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import timedelta, timezone

# mark when your session started (so we only look for *new* fills)
SESSION_START = datetime.now(timezone.utc)

def _rfc3339(dt_utc: datetime) -> str:
    """RFC3339 string for Alpaca 'after' param (must be tz-aware UTC)."""
    if dt_utc.tzinfo is None:
        dt_utc = dt_utc.replace(tzinfo=timezone.utc)
    else:
        dt_utc = dt_utc.astimezone(timezone.utc)
    # Alpaca accepts ISO8601/RFC3339 with 'Z'
    return dt_utc.isoformat().replace("+00:00", "Z")

def get_fills_since(api, symbol: str, since_utc: datetime):
    """
    Return a list of fills for `symbol` since `since_utc`.
    Each item: dict(symbol, side, qty, price, ts[UTC]).
    """
    try:
        acts = api.get_activities(
            activity_types="FILL",
            after=_rfc3339(since_utc)
        )
    except Exception as e:
        logging.warning("get_activities(FILL) failed: %s", e)
        return []

    fills = []
    for a in acts or []:
        sym = getattr(a, "symbol", None) or getattr(a, "order_symbol", None)
        if str(sym or "") != symbol:
            continue
        t_raw = (
            getattr(a, "transaction_time", None)
            or getattr(a, "filled_at", None)
            or getattr(a, "activity_time", None)
        )
        try:
            ts = pd.to_datetime(t_raw, utc=True)
        except Exception:
            continue
        fills.append({
            "symbol": symbol,
            "side":   (getattr(a, "side", "") or "").lower(),  # 'buy'/'sell'
            "qty":    float(getattr(a, "qty", 0) or 0),
            "price":  float(getattr(a, "price", np.nan)) if hasattr(a, "price") else np.nan,
            "ts":     ts
        })

    fills.sort(key=lambda x: x["ts"])
    return fills

def _bars_between(api, symbol: str, start_utc: pd.Timestamp, end_utc: pd.Timestamp) -> pd.DataFrame:
    """
    Fetch minute bars in [start, end] and return a normalized OHLCV DF.
    Uses your helper get_recent_bars if available; else falls back to direct API.
    """
    # Try your helper first (if present)
    try:
        # Heuristic: fetch enough minutes to cover the window
        mins = max(30, int((end_utc - start_utc).total_seconds() // 60) + 5)
        df = get_recent_bars(api, symbol, limit=mins, timeframe=TimeFrame.Minute)
        # Filter to the exact window just in case
        df = df.loc[(df.index >= start_utc) & (df.index <= end_utc)]
        if not df.empty:
            return df
    except Exception:
        pass

    # Fallback: direct call with start/end
    bars = api.get_bars(
        symbol,
        TimeFrame.Minute,
        start=start_utc.isoformat().replace("+00:00", "Z"),
        end=end_utc.isoformat().replace("+00:00", "Z"),
    )
    if hasattr(bars, "df"):
        df = bars.df.copy()
        if isinstance(df.index, pd.MultiIndex):
            df = df.xs(symbol, level=0)
        df.index = pd.to_datetime(df.index, utc=True)
        df = df.rename(columns={"open":"Open","high":"High","low":"Low","close":"Close","volume":"Volume"})
        return df[["Open","High","Low","Close","Volume"]].sort_index()
    else:
        rows = []
        for b in bars:
            rows.append({
                "timestamp": pd.to_datetime(getattr(b, "t", None), utc=True),
                "Open":  float(getattr(b, "o", getattr(b, "open", np.nan))),
                "High":  float(getattr(b, "h", getattr(b, "high", np.nan))),
                "Low":   float(getattr(b, "l", getattr(b, "low", np.nan))),
                "Close": float(getattr(b, "c", getattr(b, "close", np.nan))),
                "Volume":float(getattr(b, "v", getattr(b, "volume", np.nan))),
            })
        return pd.DataFrame(rows).set_index("timestamp").sort_index()

def plot_live_trades(api, symbol: str, since_utc: datetime, window_before="30min", window_after="30min"):
    """
    Plot minute price around *new* fills only.
    If no fills -> prints a note and returns (no plot).
    """
    fills = get_fills_since(api, symbol, since_utc)
    if not fills:
        print(f"No new fills for {symbol}; not plotting.")
        return

    # Build a single price series covering all fills + padding windows
    first_ts = min(f["ts"] for f in fills)
    last_ts  = max(f["ts"] for f in fills)
    start = first_ts - pd.Timedelta(window_before)
    end   = last_ts  + pd.Timedelta(window_after)

    df = _bars_between(api, symbol, start, end)
    if df.empty:
        print(f"No bars available around fills for {symbol}; not plotting.")
        return

    # Plot
    tz = "America/New_York"
    idx_ny = df.index.tz_convert(tz)
    close = df["Close"]

    plt.figure(figsize=(12, 5))
    plt.plot(idx_ny, close.values, lw=2, label=f"{symbol} Close")

    # annotate each fill
    for f in fills:
        ts_ny = f["ts"].tz_convert(tz)
        # nearest bar price for marker
        try:
            near_idx = close.index.get_indexer([f["ts"]], method="nearest")[0]
            y = close.iloc[near_idx]
        except Exception:
            y = np.nan

        if f["side"] == "buy":
            plt.scatter([ts_ny], [y], marker="^", s=80, label="BUY fill", zorder=5)
        elif f["side"] == "sell":
            plt.scatter([ts_ny], [y], marker="v", s=80, label="SELL fill", zorder=5)
        else:
            plt.scatter([ts_ny], [y], s=70, label="FILL", zorder=5)

        # optional dashed vline
        plt.axvline(ts_ny, linestyle="--", alpha=0.2)

    plt.title(f"{symbol} — Live fills")
    plt.grid(True, alpha=0.3)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

In [22]:
# --- Trade & Position Summary (UNH-only) ---
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Report only what's in your env (default UNH)
tickers_to_report = [t.strip().upper() for t in os.getenv("TICKERS", "UNH").split(",") if t.strip()]

# Where your trade logs live; default to ARTIFACTS_DIR
ART_DIR = Path(os.getenv("ARTIFACTS_DIR", "/content"))
RESULTS_DIR = Path(os.getenv("RESULTS_DIR", str(ART_DIR)))

print("\nTrade Summary:")
for ticker in tickers_to_report:
    log_path = RESULTS_DIR / f"trade_log_{ticker}.csv"
    if not log_path.exists():
        print(f"{ticker}: no trades logged yet.")
        continue

    try:
        df = pd.read_csv(log_path, on_bad_lines="skip",
                         parse_dates=["log_time","bar_time"], infer_datetime_format=True)

        # Count by 'signal' (or 'action' if older logs)
        key = "signal" if "signal" in df.columns else ("action" if "action" in df.columns else None)
        if key:
            counts = df[key].value_counts(dropna=False).to_dict()
            print(f"{ticker}: {counts}")
        else:
            print(f"{ticker}: log present but missing 'signal'/'action' columns.")

        # Optional confidence histogram
        if "confidence" in df.columns and df["confidence"].notna().any():
            plt.figure(figsize=(8, 3.5))
            df["confidence"].dropna().plot(kind="hist", bins=10, edgecolor="black")
            plt.title(f"{ticker} - Confidence Distribution")
            plt.xlabel("confidence")
            plt.tight_layout()
            plt.show()

        # Optional: PPO-specific stats
        for col in ["weight", "raw_action"]:
            if col in df.columns and df[col].notna().any():
                s = df[col].dropna()
                print(f"{ticker} {col}: mean={s.mean():.3f}, std={s.std():.3f}, "
                      f"min={s.min():.3f}, max={s.max():.3f}")

    except Exception as e:
        print(f"{ticker}: could not summarize trades: {e}")

# --- Position Summary (unchanged) ---
try:
    if 'api' not in globals():
        api = init_alpaca()
    positions = api.list_positions()
    total_market_value = 0.0
    print("\nPosition Summary:")
    for p in positions:
        mv = float(p.market_value)
        total_market_value += mv
        print(f"  {p.symbol}: {p.qty} shares @ ${float(p.current_price):.2f} | Value: ${mv:,.2f}")
    print(f"\nTotal Market Value: ${total_market_value:,.2f}")
except Exception as e:
    print(f"Could not summarize positions: {e}")



Trade Summary:
UNH: no trades logged yet.

Position Summary:
  AAPL: 2 shares @ $234.35 | Value: $468.70
  ABT: 1 shares @ $131.95 | Value: $131.95
  BRK.B: 1 shares @ $492.81 | Value: $492.81
  MDT: 1 shares @ $94.07 | Value: $94.07
  PG: 1 shares @ $159.50 | Value: $159.50
  PM: 1 shares @ $164.74 | Value: $164.74

Total Market Value: $1,511.77
